# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

d:\Courses\Data Engineering\project_2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
           
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [6]:
query = """
    create keyspace if not exists udacity
    with replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Task 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For this query, the data model for it will be like this:

Table name: session_history

Description: Store history for sessions and item in each session

| column name   | data type | is primary key? | type of primary key | description |
|---------------|-----------|-----------------|---------------------|-------------|
| sessionId     | int       | yes             | partition           | unique identifier for a session |
| itemInSession | int       | yes             | partition           | unique identifier for an item in a session |
| artist        | text      |                 |                     | the artist name |
| song          | text      |                 |                     | the song title |
| length        | float     |                 |                     | the length of the song in seconds |

For this query, sessionId and itemInSession will be created as composite key because the query wants the filtering for these two columns so it must be in the composite key for primary key.


In [8]:
query = """
    create table if not exists session_history (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        primary key (sessionId, itemInSession)
    );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into session_history (sessionId, itemInSession, artist, song, length) "
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Get artist name, song name, and song length with id of session is 338 and item in session is 4
query = """
    select artist, song, length
    from session_history
    where sessionId = 338 and itemInSession = 4
"""
try:
    rows = session.execute(query)
    for row in rows:
        print("artist: {}\nsong: {}\nlength: {}\n---------------------\n".format(row.artist, row.song, row.length))
except Exception as e:
    print(e)

artist: Faithless
song: Music Matters (Mark Knight Dub)
length: 495.30731201171875
---------------------



### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query, the data model for it will be like this:

Table name: user_session_history

Description: Store history of sessions for users

| column name   | data type | is primary key? | type of primary key | description |
|---------------|-----------|-----------------|---------------------|-------------|
| userId        | int       | yes             | partition           | unique identifier for a user |
| sessionId     | int       | yes             | partition           | unique identifier for a session |
| itemInSession | int       | yes             | clustering          | unique identifier for an item in a session, used for storing song plays |
| artist        | text      |                 |                     | the artist name |
| song          | text      |                 |                     | the song title |
| firstName     | text      |                 |                     | the user's first name |
| lastName      | text      |                 |                     | the user's last name |

For this query, userId and sessionId will be created as composite key because the query wants the filtering for these two columns so it must be in the composite key for primary key. And for the itemInSession column, it will be the clustering key for this table because the query want the itemInSession to be sorted so it will be sorted for each data partitioned by userId and sessionId.

In [11]:
query = """
    create table if not exists user_session_history (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        primary key ((userId, sessionId), itemInSession)
    );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into user_session_history (userId, sessionId, artist, song, firstName, lastName, itemInSession) "
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

In [13]:
# Get artist name, song name, first and last name of user with id of 10 and session id of him/her is 182
query = """
    select artist, song, firstname, lastname
    from user_session_history
    where userId = 10 and sessionId = 182
"""
try:
    rows = session.execute(query)
    for row in rows:
        print("artist: {}\nsong: {}\nfirstname: {}\nlastname: {}\n----------------------\n"
            .format(row.artist, row.song, row.firstname, row.lastname))
except Exception as e:
    print(e)

artist: Down To The Bone
song: Keep On Keepin' On
firstname: Sylvie
lastname: Cruz
----------------------

artist: Three Drives
song: Greece 2000
firstname: Sylvie
lastname: Cruz
----------------------

artist: Sebastien Tellier
song: Kilometer
firstname: Sylvie
lastname: Cruz
----------------------

artist: Lonnie Gordon
song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
firstname: Sylvie
lastname: Cruz
----------------------



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query, the data model for it will be like this:

Table name: user_history

Description: Store history of songs played by users

| column name | data type | is primary key? | type of primary key | description |
|-------------|-----------|-----------------|---------------------|-------------|
| song        | text      | yes             | partition           | unique identifier for a song |
| userId      | int       | yes             | partition           | unique identifier for a user |
| firstName   | text      |                 |                     | the user's first name |
| lastName    | text      |                 |                     | the user's last name |

For this query, song and userId will be created as composite key because the query wants the filtering for song and a song can be played by many users so to avoid duplicate, userId must be added to create a composite key with song.

In [14]:
query = """
    create table if not exists user_history (
        song text,
        userId int,
        firstName text,
        lastName text,
        primary key (song, userId)
    );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into user_history (song, userId, firstName, lastName) "
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# Get first and last name of users that play song 'All Hands Against His Own'
query = """
    select firstname, lastname
    from user_history
    where song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query)
    for row in rows:
        print("Firstname: {}\nLastname: {}\n-----------------------\n".format(row.firstname, row.lastname))
except Exception as e:
    print(e)

Firstname: Jacqueline
Lastname: Lynch
-----------------------

Firstname: Tegan
Lastname: Levine
-----------------------

Firstname: Sara
Lastname: Johnson
-----------------------



### Drop the tables before closing out the sessions

In [18]:
def drop_table(table_name):
    query = ("drop table if exists %s;" % table_name)
    try:
        session.execute(query)
    except Exception as e:
        print(e)

drop_table('user_history')

drop_table('user_session_history')

drop_table('session_history')

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()